In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import os
import feather
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error 
import random

In [3]:
import sys
if "/proj/yunligrp/users/minzhi/utils/pylib" not in sys.path:
    sys.path.insert(0, "/proj/yunligrp/users/minzhi/utils/pylib")

In [4]:
from function_process_data_eqtl import *
from function_asso import *
from function_mesa_cca import *
%autoreload 2

# I. Alpha Globin

## 1. cohort and 2. # samples with calls

In [432]:
cn_pass_useful = cn_pass[["SAMPLE", "CN"]]
cn_pass_useful.rename(columns = {"SAMPLE":"NWDID", "CN":"cn"}, inplace = True)
cn_pass_dir = os.path.join("..", "prepro_data", "cn")
cn_pass_filename = "alpha_globin_calls_pass_useful.tsv"
cn_pass_dir_filename = os.path.join(cn_pass_dir, cn_pass_filename)
cn_pass_useful.to_csv(cn_pass_dir_filename, sep = "\t", header = True, index = False)

### 2.2 Summary of Annotation

In [44]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_sample_annot_2019-10-08_useful_unique02.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

In [49]:
cn_annotation = cn_pass.merge(annotation, left_on = "SAMPLE", right_on = "NWDID", how = "inner")
cn_pass_anno_summary, cn_pass_anno_list = categorize_df(cn_annotation, "study")
print(cn_annotation.shape)
print(cn_pass_anno_list)
cn_pass_anno_summary_dir = os.path.join("..", "data_summary")
cn_pass_anno_summary_filename = "cn-nathan_pass_anno_cohort_summary.tsv"
cn_pass_anno_summary_dir_filename = os.path.join(cn_pass_anno_summary_dir, cn_pass_anno_summary_filename)
cn_pass_anno_summary.to_csv(cn_pass_anno_summary_dir_filename, sep = "\t", header = True, index = False)

(118934, 15)
['GOLDN', 'INSPIRE_AF', 'CAMP', 'GALAI', 'VU_AF', 'HCHS_SOL', 'CARDIA', 'EGCUT', 'THRV', 'PUSH_SCD', 'DECAF', 'REDS-III_Brazil', 'MGH_AF', 'AustralianFamilialAF', 'IPF', 'OMG_SCD', 'Partners', 'CHIRAH', 'BioMe', 'SAS', 'BioVU_AF', 'GENOA', 'CARE_PACT', 'CARE_BADGER', 'PharmHU', 'PCGC_CHD', 'CFS', 'GenSalt', 'PIMA', 'Sarcoidosis', 'HyperGEN', 'BAGS', 'PMBB_AF', 'WHI', 'walk_PHaSST', 'EOCOPD', 'GGAF', 'LTRC', 'WGHS', 'SARP', 'MPP', 'CARE_CLIC', 'MLOF', 'FHS', 'ARIC', 'ChildrensHS_IGERA', 'HVH', 'GeneSTAR', 'COPDGene', 'CCAF', 'SAFS', 'GENAF', 'SAGE', 'GALAII', 'DHS', 'SAPPHIRE_asthma', 'AFLMU', 'CATHGEN', 'MESA', 'VAFAR', 'miRhythm', 'JHS', 'CRA', 'JHU_AF', 'Mayo_VTE', 'CHS', 'Amish', 'ChildrensHS_MetaAir', 'CARE_TREXA', 'ChildrensHS_GAP', 'ECLIPSE']


## 3. # samples with phenotype data

### DDIMER

In [18]:
ddimer_filename = "TOPMED_HarmonizedPhenotypes_DDIMER_21MAY2019.csv"
ddimer_dir = os.path.join("..", "raw_data", "phenotype")
ddimer_dir_filename = os.path.join(ddimer_dir, ddimer_filename)
ddimer_raw = pd.read_csv(ddimer_dir_filename, sep = ",", header = 0, index_col = None)
ddimer = ddimer_raw.loc[:, ["sample.id", "STUDY", "DDIMER", "AGE_DDIMER", "sample_remove_DDIMER"]]
ddimer.dropna(axis = "index", how = "any", inplace = True)
ddimer_category_summary, _ = categorize_df(ddimer, "STUDY")
ddimer_category_summary.sort_values(by = "cohort", axis = 0, inplace = True)
ddimer_category_summary_dir = os.path.join("..", "data_summary")
ddimer_category_summary_filename = "TOPMED_HarmonizedPhenotypes_DDIMER_21MAY2019_study_summary.tsv"
ddimer_category_summary_dir_filename = os.path.join(ddimer_category_summary_dir, ddimer_category_summary_filename)
ddimer_category_summary.to_csv(ddimer_category_summary_dir_filename, sep = "\t", header = True, index = False)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
ddimer_qc = ddimer[ddimer["sample_remove_DDIMER"] != 1]
ddimer_qc_category_summary, _ = categorize_df(ddimer_qc, "STUDY")
ddimer_qc_category_summary.sort_values(by = "cohort", axis = 0, inplace = True)
ddimer_qc_category_summary_dir = os.path.join("..", "data_summary")
ddimer_qc_category_summary_filename = "TOPMED_HarmonizedPhenotypes_DDIMER_21MAY2019_qc_study_summary.tsv"
ddimer_qc_category_summary_dir_filename = os.path.join(ddimer_qc_category_summary_dir, ddimer_qc_category_summary_filename)
ddimer_qc_category_summary.to_csv(ddimer_qc_category_summary_dir_filename, sep = "\t", header = True, index = False)

### eGFR

In [10]:
egfr_dir = os.path.join("..", "raw_data", "egfr")
filename_list = os.listdir(egfr_dir)

In [20]:
cohort_list = []
sample_size_list = []
for filename in filename_list[1:]:
    cohort_name_raw = filename.split("_")[3]
    if cohort_name_raw == "PHEN":
        cohort_name = "SOL"
    else:
        cohort_name = cohort_name_raw
    dir_filename = os.path.join(egfr_dir, filename)
    tmp_df = pd.read_csv(dir_filename, sep = "\t", header = 0, index_col = None)
    tmp_sample_size = tmp_df.shape[0]
    cohort_list.append(cohort_name)
    sample_size_list.append(tmp_sample_size)
cohort_sample_size_tuple = list(zip(cohort_list, sample_size_list))
egfr_sample_size_df = pd.DataFrame(data = cohort_sample_size_tuple, columns = ["cohort", "sample_size"])
egfr_sample_size_df.sort_values(by = "cohort", axis = 0, inplace = True)
egfr_sample_size_dir = os.path.join("..", "data_summary")
egfr_sample_size_filename = "egfr_cohort_summary.tsv"
egfr_sample_size_dir_filename = os.path.join(egfr_sample_size_dir, egfr_sample_size_filename)
egfr_sample_size_df.to_csv(egfr_sample_size_dir_filename, sep = "\t", header = True, index = False)

### grengrp6

In [24]:
gengrp6_filename = "page-harmonized-phenotypes-pca-freeze2-candidate2-2016-12-14.GWASid_fid_22May2018internalPCs.SOLv2consent.txt"
gengrp6_dir = os.path.join("..", "raw_data", "adjustment", "gengrp6")
gengrp6_dir_filename = os.path.join(gengrp6_dir, gengrp6_filename)
gengrp6 = pd.read_csv(gengrp6_dir_filename, sep = "\t", header = 0, index_col = None)
gengrp6.replace(".", np.nan, inplace=True)
gengrp6_select = gengrp6[["z_sol_id", "analysis_id", "CONSENT_text", "INTERNAL_USE_ONLY",
                          "gengrp6"]].dropna(axis=0, subset=["analysis_id","gengrp6"],how="any")
gengrp6_select.rename(columns = {"analysis_id":"SUBJECT_ID"}, inplace=True)
print(gengrp6_select.shape)
gengrp6_select = gengrp6_select[gengrp6_select["CONSENT_text"] != "DROP"]
print(gengrp6_select.shape)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,28,29,30,35,39,42,45,46,47,49,54,56,61,63,69,70,71,72,73,76,83,85,89,90,91,98,114,117,125,132,134,135,142,155,156,160,161,163,164,176,186,187,192,196,205,206,207,208,209,210,216,218,220,224,225,228,234,242,244,247,250,260,269,270,272,282,283,285,286,288,292,294,295,296,298,303,305,335,336,383,385,386,389,390,391,392,393) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(11829, 5)
(11678, 5)


### weight and center

In [27]:
weight_center_dir = os.path.join("..", "raw_data", "adjustment")
weight_center_filename = "bloodcell_output.csv"
weight_center_dir_filename = os.path.join(weight_center_dir, weight_center_filename)
weight_center = pd.read_csv(weight_center_dir_filename, sep = ",", header = 0, index_col = None)
weight_center_select = weight_center[["ID", "WEIGHT_FINAL_NORM_OVERALL", "CENTER"]].dropna(axis = 0, how = "any")
weight_center_select.rename(columns = {"ID":"z_sol_id"}, inplace = True)
print(weight_center_select.shape)
gengrp6_weight_center = gengrp6_select.merge(weight_center_select, on = "z_sol_id", how = "inner")
print(gengrp6_weight_center.shape)

(16415, 3)
(9974, 7)


### blood cell traits

In [ ]:
pheno_dir = os.path.join("..", "raw_data", "phenotype")
pheno_filename = "coh02_pre.tsv"
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)
cat_col = "cohort"
pheno_cat_summary, cohort_list = categorize_df(pheno, cat_col)
cat_list = cohort_list
col_list = ["hemoglobin_mcnc_bld_1", "hematocrit_vfr_bld_1", "rbc_ncnc_bld_1", "mcv_entvol_rbc_1", "mch_entmass_rbc_1", "mchc_mcnc_rbc_1",
            "rdw_ratio_rbc_1", "neutrophil_ncnc_bld_1", "lymphocyte_ncnc_bld_1", "basophil_ncnc_bld_1", "eosinophil_ncnc_bld_1",
            "monocyte_ncnc_bld_1", "wbc_ncnc_bld_1", "pmv_entvol_bld_1", "platelet_ncnc_bld_1"]
fixed_col_list = ["SUBJECT_ID", "unique_subject_key", "cohort"]
cohort_pheno_df = cat_col_summary(pheno, cat_col, cat_list, col_list, fixed_col_list)
cohort_pheno_dir = os.path.join("..", "data_summary")
cohort_pheno_filename = "cohort_pheno_summary.tsv"
cohort_pheno_dir_filename = os.path.join(cohort_pheno_dir, cohort_pheno_filename)
cohort_pheno_df.to_csv(cohort_pheno_dir_filename, sep = "\t", header = True, index = True)

## 4. # sample in annotation file

### 4.1 Cohort Summary

In [94]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_sample_annot_2019-07-30.txt"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
annotation.dropna(axis = 0, subset = ["study"], how = "any", inplace = True)
annotation_cat_summary, _ = categorize_df(annotation, "study")
annotation_cat_summary_dir = os.path.join("..", "data_summary")
annotation_cat_summary_filename = "annotation_cohort_summary_2019-07-30.tsv"
annotation_cat_summary_dir_filename = os.path.join(annotation_cat_summary_dir, annotation_cat_summary_filename)
annotation_cat_summary.to_csv(annotation_cat_summary_dir_filename, sep = "\t", header = True, index = False)

### 4.2 Consent Summary

In [104]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_sample_annot_2019-07-30.txt"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
sample_size = annotation.shape[0]
annotation.dropna(axis = 0, subset = ["consent"], how = "any", inplace = True)
sample_size_nona = annotation.shape[0]
na_df = pd.DataFrame(data = [["NAN", sample_size - sample_size_nona]], columns = ["consent", "sample_size"])
annotation_cat_summary, _ = categorize_df(annotation, "consent")
annotation_cat_summary = annotation_cat_summary.append(na_df, ignore_index=True)
annotation_cat_summary_dir = os.path.join("..", "data_summary")
annotation_cat_summary_filename = "annotation_consent_summary_2019-07-30.tsv"
annotation_cat_summary_dir_filename = os.path.join(annotation_cat_summary_dir, annotation_cat_summary_filename)
annotation_cat_summary.to_csv(annotation_cat_summary_dir_filename, sep = "\t", header = True, index = False)

### 4.3 Consent Summary by Cohorts

In [17]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_sample_annot_2019-10-08.txt"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
annotation.dropna(axis = 0, subset = ["study"], how = "any", inplace = True)
_, cohort_list = categorize_df(annotation, "study")

In [19]:
consent_summary_cohort = pd.DataFrame(columns=['cohort', 'consent', 'sample_size'])
for cohort in cohort_list:
    tmp_annotation = annotation.loc[annotation.loc[:, "study"] == cohort, :]
    tmp_summary, _ = categorize_df(tmp_annotation, "consent")
    row_num = tmp_summary.shape[0]
    tmp_cohort_series = pd.Series(data = [cohort] * row_num)
    tmp_summary.insert(loc = 0, column = "cohort", value = tmp_cohort_series)
    consent_summary_cohort = pd.concat([consent_summary_cohort, tmp_summary], axis = 0, ignore_index = True)

In [20]:
consent_summary_cohort_dir = os.path.join("..", "data_summary")
consent_summary_cohort_filename = "annotation_consent_summary_cohort_2019-10-08.tsv"
consent_summary_cohort_dir_filename = os.path.join(consent_summary_cohort_dir, consent_summary_cohort_filename)
consent_summary_cohort.to_csv(consent_summary_cohort_dir_filename, sep = "\t", header = True, index = False)

### 4.4 Clean Annotation File

In [32]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_sample_annot_2020-03-03.txt"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation_raw = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
annotation_useful_header_list = ["sample.id", "unique_subject_key", "subject_id", "consent", "study", "sex", "exclude"]
annotation = annotation_raw[annotation_useful_header_list]
annotation.dropna(axis = 0, how = "any", inplace = True)
annotation.rename(columns = {"sample.id":"NWDID"}, inplace = True)
annotation.loc[annotation.loc[:, "sex"] == "M", "sex"] = 1
annotation.loc[annotation.loc[:, "sex"] == "F", "sex"] = 0

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
annotation = annotation.loc[annotation.loc[:, "exclude"] == False, :]
annotation.reset_index(inplace = True, drop = True)
sample_num = annotation.shape[0]
consent_remove_idx = []
cohort_list = ["ARIC", "COPDGene", "CHS"]
for sample_i in range(sample_num):
    tmp_consent = annotation.loc[sample_i, "consent"]
    tmp_cohort = annotation.loc[sample_i, "study"]
    tmp_consent_list = tmp_consent.split("-")
    if (tmp_cohort in cohort_list) and (tmp_consent_list[0] == "DS"):
        consent_remove_idx.append(sample_i)
annotation_remove_consent = annotation.drop(axis = 0, labels = consent_remove_idx)

In [34]:
annotation_remove_consent_filename = "freeze8_anno05_af02.tsv"
annotation_remove_consent_dir_filename = os.path.join(annotation_dir, annotation_remove_consent_filename)
annotation_remove_consent.to_csv(annotation_remove_consent_dir_filename, sep = "\t", header = True, index = False)

### 4.5 Remove Duplicates from ToPMed Duplicates List

In [35]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
dup_list_filename = "freeze8_duplicates_2019-04-19.txt"
dup_list_dir_filename = os.path.join(annotation_dir, dup_list_filename)
dup_list = pd.read_csv(dup_list_dir_filename, sep = "\t", header = 0, index_col = None)
dup_list.sort_values(by = ["study1"], inplace = True)

annotation_filename = "freeze8_anno05_af02.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

In [36]:
dup_list_control = dup_list.loc[dup_list.loc[:, "study1"] == "Control", :]
dup_list_control_list = dup_list_control["ID2"].values.tolist()

In [37]:
dup_list_norm = dup_list.loc[dup_list.loc[:, "study1"] != "Control", :]
norm_num = dup_list_norm.shape[0]
np.random.seed(43329)
norm_idx_list = np.random.randint(2, size=norm_num)
dup_list_norm_list = []
for sample_i in range(norm_num):
    norm_idx = norm_idx_list[sample_i]
    tmp_sample_id = dup_list_norm.iloc[sample_i, norm_idx]
    dup_list_norm_list.append(tmp_sample_id)

In [38]:
dup_id_list = dup_list_control_list + dup_list_norm_list
print(annotation.shape)
annotation_unique = annotation[~annotation['NWDID'].isin(dup_id_list)]
print(annotation_unique.shape)
annotation_unique_filename = "freeze8_anno05_af02_unique01.tsv"
annotation_unique_dir_filename = os.path.join(annotation_dir, annotation_unique_filename)
annotation_unique.to_csv(annotation_unique_dir_filename, sep = "\t", header = True, index = False)

(138092, 7)
(136047, 7)


### 4.6 Duplicates of unique_subject_key in freeze8 annotation file

In [39]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_anno05_af02_unique01.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

annotation_dup_df = df_extraction_duplicates(annotation, "unique_subject_key")
print(annotation_dup_df.shape)
annotation_dup_df = annotation_dup_df.sort_values(by=['unique_subject_key'])
display(annotation_dup_df)

annotation_dup_filename = "freeze8_anno05_af02_unique01_dup.tsv"
annotation_dup_dir_filename = os.path.join(annotation_dir, annotation_dup_filename)
annotation_dup_df.to_csv(annotation_dup_dir_filename, sep = "\t", header = True, index = False)

(0, 7)


,NWDID,unique_subject_key,subject_id,consent,study,sex,exclude


In [40]:
invar_subsets = ["consent", "sex"]
rm_header = "NWDID"
annotation_unique = remove_dup_anno(annotation, invar_subsets, rm_header)
annotation_unique_filename = "freeze8_anno05_af02_unique02.tsv"
annotation_unique_dir_filename = os.path.join(annotation_dir, annotation_unique_filename)
annotation_unique.to_csv(annotation_unique_dir_filename, sep = "\t", header = True, index = False)

### 4.7 Duplicates of NWDID in freeze8 annotation file

In [41]:
annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_anno05_af02_unique02.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
print(duplicates_num(annotation, "NWDID"))

0


### 4.8 Process the overlap between 3 & 4

In [96]:
annotation_dir = os.path.join("..", "raw_data", "phenotype")
annotation_filename = "freeze8_sample_annot_2019-05-30.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
annotation_drop_subject_key = annotation.dropna(axis = 0, subset = ["unique_subject_key"], how = "any")
print(annotation_drop_subject_key.shape)
annotation_drop_subject_id = annotation.dropna(axis = 0, subset = ["subject_id"], how = "any")
print(annotation_drop_subject_id.shape)
annotation_drop_study = annotation.dropna(axis = 0, subset = ["study"], how = "any")
print(annotation_drop_study.shape)
annotation.dropna(axis = 0, subset = ["unique_subject_key", "subject_id", "NWDID"], how = "any", inplace = True)
print(annotation.shape)
annotation_select = annotation[["NWDID", "subject_id", "unique_subject_key", "consent", "study", "sex", "exclude"]]
annotation_select.rename(columns = {"subject_id":"SUBJECT_ID", "study":"cohort"}, inplace = True)
annotation_select_dir = os.path.join("..", "raw_data", "phenotype")
annotation_select_filename = "freeze8_sample_annot_2019-05-30_useful.tsv"
annotation_select_dir_filename = os.path.join(annotation_select_dir, annotation_select_filename)
#annotation_select.to_csv(annotation_select_dir_filename, sep = "\t", header = True, index = False)

(133280, 18)
(133280, 18)
(138934, 18)
(133280, 18)


## 5. Overlap b/w 3 & 4

In [22]:
pheno_filename = "coh03_pheno02_pre.tsv"
pheno_dir = os.path.join("..", "raw_data", "phenotype")
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)

annotation_dir = os.path.join("..", "raw_data", "phenotype")
annotation_filename = "freeze8_sample_annot_2019-05-30_useful.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
annotation.drop(axis = 1, labels = ["SUBJECT_ID", "cohort"], inplace = True)

pheno_annotation = pheno.merge(annotation, on = ["unique_subject_key"], how = "inner")
print(pheno_annotation.shape)
pheno_annotation.dropna(axis = 0, subset = ["cohort"], inplace = True)
print(pheno_annotation.shape)
print(duplicates_num(pheno_annotation, "NWDID"))
print("There is no duplicates in NWDID in the merged table of phenotype and annotation.")
pheno_annotation_dir = os.path.join("..", "raw_data", "phenotype")
pheno_annotation_filename = "coh03_pheno02_freeze8_anno_pre.tsv"
pheno_annotation_dir_filename = os.path.join(pheno_annotation_dir, pheno_annotation_filename)
pheno_annotation.to_csv(pheno_annotation_dir_filename, sep = "\t", header = True, index = False)

(66696, 47)
(66696, 47)
0
There is no duplicates in NWDID in the merged table of phenotype and annotation.


In [28]:
pheno_filename = "coh03_pheno02_pre.tsv"
pheno_dir = os.path.join("..", "raw_data", "phenotype")
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)

annotation_dir = os.path.join("..", "raw_data", "phenotype")
annotation_filename = "freeze8_sample_annot_2019-05-30_useful.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

pheno_annotation_subjectid_uni = pheno.merge(annotation, on = ["SUBJECT_ID", "unique_subject_key"], how = "inner")
print(pheno_annotation_subjectid_uni.shape)
pheno_annotation_subjectid_cohort_uni = pheno.merge(annotation, on = ["SUBJECT_ID", "unique_subject_key", "cohort"], how = "inner")
print(pheno_annotation_subjectid_cohort_uni.shape)
pheno_annotation_subjectid_cohort = pheno.merge(annotation, on = ["SUBJECT_ID", "cohort"], how = "inner")
print(pheno_annotation_subjectid_cohort.shape)
pheno_annotation_subjectid = pheno.merge(annotation, on = ["SUBJECT_ID"], how = "inner")
print(pheno_annotation_subjectid.shape)
pheno_annotation_uni = pheno.merge(annotation, on = ["unique_subject_key"], how = "inner")
print(pheno_annotation_uni.shape)

annotation.drop(axis = 1, labels = ["SUBJECT_ID", "cohort"], inplace = True)

pheno_annotation = pheno.merge(annotation, on = ["unique_subject_key"], how = "inner")
print(pheno_annotation.shape)
pheno_annotation.dropna(axis = 0, subset = ["cohort"], inplace = True)
print(pheno_annotation.shape)
print(duplicates_num(pheno_annotation, "NWDID"))
print("There is no duplicates in NWDID in the merged table of phenotype and annotation.")
pheno_annotation_dir = os.path.join("..", "raw_data", "phenotype")
pheno_annotation_filename = "coh03_pheno02_freeze8_anno_pre.tsv"
pheno_annotation_dir_filename = os.path.join(pheno_annotation_dir, pheno_annotation_filename)

(46105, 48)
(46105, 47)
(46105, 48)
(55366, 49)
(66696, 49)
(66696, 47)
(66696, 47)
0
There is no duplicates in NWDID in the merged table of phenotype and annotation.


In [120]:
load_dir = os.path.join("..", "raw_data", "phenotype")
ddimer_filename = "DDIMER_21MAY2019_complete_useful_col.tsv"
ddimer_dir_filename = os.path.join(load_dir, ddimer_filename)
ddimer = pd.read_csv(ddimer_dir_filename, sep = "\t", header = 0, index_col = None)

pheno_annotation_filename = "coh03_pheno02_freeze8_anno_pre.tsv"
pheno_annotation_dir_filename = os.path.join(load_dir, pheno_annotation_filename)
pheno_annotation = pd.read_csv(pheno_annotation_dir_filename, sep = "\t", header = 0, index_col = None)

pheno_annotation_ddimer_comparesex = pheno_annotation.merge(ddimer, on = ["NWDID", "cohort", "sex"], how = "outer")
print(duplicates_num(pheno_annotation_ddimer_comparesex, "NWDID"))
pheno_annotation_ddimer_nocomparesex = pheno_annotation.merge(ddimer, on = ["NWDID", "cohort"], how = "outer")
print(duplicates_num(pheno_annotation_ddimer_nocomparesex, "NWDID"))

boolean_series = pheno_annotation_ddimer_comparesex[["NWDID"]].duplicated(keep=False)
pheno_annotation_ddimer_comparesex_dup = pheno_annotation_ddimer_comparesex.loc[boolean_series, :]
pheno_annotation_ddimer_comparesex_dup.sort_values(axis = 0, by = "NWDID", inplace = True)

ddimer_anno_sex_diff_dir = os.path.join("..", "data_summary")
ddimer_anno_sex_diff_filename = "ddimer_anno_sex_diff.tsv"
ddimer_anno_sex_diff_dir_filename = os.path.join(ddimer_anno_sex_diff_dir, ddimer_anno_sex_diff_filename)
pheno_annotation_ddimer_comparesex_dup.to_csv(ddimer_anno_sex_diff_dir_filename, sep = "\t", header = True, index = False)

display(pheno_annotation_ddimer_comparesex_dup)

4
0


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,SUBJECT_ID,unique_subject_key,cohort,hemoglobin_mcnc_bld_1,age_at_hemoglobin_mcnc_bld_1,hematocrit_vfr_bld_1,age_at_hematocrit_vfr_bld_1,rbc_ncnc_bld_1,age_at_rbc_ncnc_bld_1,mcv_entvol_rbc_1,...,gengrp6,WEIGHT_FINAL_NORM_OVERALL,CENTER,NWDID,consent,sex,exclude,age_at_DDIMER,DDIMER,sample_remove_DDIMER
53932,131150792,ARIC_131150792,ARIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD355569,HMB-IRB,M,True,NaN,NaN,NaN
68851,NaN,NaN,ARIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD355569,NaN,F,NaN,61.0,48000.0,0.0
52892,131186968,ARIC_131186968,ARIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD355606,HMB-IRB,M,True,NaN,NaN,NaN
69022,NaN,NaN,ARIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD355606,NaN,F,NaN,57.0,25000.0,0.0
59496,9229,FHS_9229,FHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD611521,HMB-IRB-NPU-MDS,F,False,NaN,NaN,NaN
68671,NaN,NaN,FHS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD611521,NaN,M,NaN,61.0,23500.0,0.0
31155,823218,WHI_823218,WHI,13.2,78.25,39.9,78.25,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD632820,HMB-IRB-NPU,M,False,NaN,NaN,NaN
69026,NaN,NaN,WHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NWD632820,NaN,F,NaN,78.0,25220.0,0.0


Manually remove the samples with different sex but the same NWDID.

In [123]:
pheno_annotation_ddimer = pheno_annotation_ddimer_comparesex.drop(labels = [53932, 52892, 59496, 31155], axis = 0)
print(pheno_annotation_ddimer.shape)
print(duplicates_num(pheno_annotation_ddimer, "NWDID"))
pheno_annotation_ddimer_dir = os.path.join("..", "raw_data", "phenotype")
pheno_annotation_ddimer_filename = "coh03_pheno03_freeze8_anno_pre.tsv"
pheno_annotation_ddimer_dir_filename = os.path.join(pheno_annotation_ddimer_dir, pheno_annotation_ddimer_filename)
pheno_annotation_ddimer.to_csv(pheno_annotation_ddimer_dir_filename, sep = "\t", header = True, index = False)

(69394, 50)
0


### Summary of phenotypes that can maps to annotation file based on cohort

In [124]:
pheno_annotation_dir = os.path.join("..", "raw_data", "phenotype")
pheno_annotation_filename = "coh03_pheno03_freeze8_anno_pre.tsv"
pheno_annotation_dir_filename = os.path.join(pheno_annotation_ddimer_dir, pheno_annotation_filename)
pheno_annotation = pd.read_csv(pheno_annotation_dir_filename, sep = "\t", header = 0, index_col = None)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (38,40,42,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [138]:
col_list = ["gengrp6", "WEIGHT_FINAL_NORM_OVERALL", "CENTER", "DDIMER", "EGFRCKDEPI", "hemoglobin_mcnc_bld_1", "hematocrit_vfr_bld_1",
            "rbc_ncnc_bld_1", "mcv_entvol_rbc_1", "mch_entmass_rbc_1", "mchc_mcnc_rbc_1", "rdw_ratio_rbc_1", "neutrophil_ncnc_bld_1",
            "lymphocyte_ncnc_bld_1", "basophil_ncnc_bld_1", "eosinophil_ncnc_bld_1", "monocyte_ncnc_bld_1", "wbc_ncnc_bld_1",
            "pmv_entvol_bld_1", "platelet_ncnc_bld_1", "lnHBA1C"]
pheno_annotation.dropna(axis = 0, subset = ["cohort"], how = "any", inplace = True)
cat_col = "cohort"
pheno_annotation_summary, category_list = categorize_df(pheno_annotation, cat_col)
pheno_annotation_category_df = pd.DataFrame(columns = col_list, index = category_list)
for category in category_list:
    for col in col_list:
        tmp_df = pheno_annotation.loc[pheno_annotation["cohort"] == category, col]
        tmp_df.dropna(axis = 0, how = "any", inplace = True)
        pheno_annotation_category_df.loc[category, col] = tmp_df.shape[0]
pheno_annotation_category_df.sort_index(axis = 0, inplace = True)
pheno_annotation_category_dir = os.path.join("..", "data_summary")
pheno_annotation_category_filename = "overlap_pheno_annotation_noexclude_summary.tsv"
pheno_annotation_category_dir_filename = os.path.join(pheno_annotation_category_dir, pheno_annotation_category_filename)
pheno_annotation_category_df.to_csv(pheno_annotation_category_dir_filename, sep = "\t", header = True, index = True)

## 6. Overlap b/w 2 & 3

Preprocess CN sample list to get list of cram of each cohort.

In [56]:
cohort_list = ["freeze6-AA", "JHS", "OMG-SCD", "PAGE", "PharmHU", "REDS-III-Brazil", "SAS", "SCD", "SOL", "walk-PHaSST"]
cohort_dir = os.path.join("..", "..", "cnv", "sample_list")
for cohort in cohort_list:
    tmp_cohort_dir_filename = os.path.join(cohort_dir, "%s_chr16_full_sample.list"%cohort)
    tmp_cohort = pd.read_csv(tmp_cohort_dir_filename, sep = "/", header = None, index_col = None)
    tmp_sample_list = tmp_cohort.iloc[:, -1].values.tolist()
    tmp_sample_id_list = []
    for tmp_sample in tmp_sample_list:
        tmp_sample_id = tmp_sample.split(".")[0]
        tmp_sample_id_list.append(tmp_sample_id)
    tmp_sample_id_df = pd.DataFrame(data = tmp_sample_id_list)
    tmp_sample_id_dir = cohort_dir
    tmp_sample_id_filename = "%s_cram.tsv"%cohort
    tmp_sample_id_dir_filename = os.path.join(tmp_sample_id_dir, tmp_sample_id_filename)
    tmp_sample_id_df.to_csv(tmp_sample_id_dir_filename, sep = "\t", header = False, index = False)

### Overlap b/w samples we have CN and pheno_anno got in section 5

In [155]:
pheno_annotation_dir = os.path.join("..", "raw_data", "phenotype")
pheno_annotation_filename = "coh03_pheno03_freeze8_anno_pre.tsv"
pheno_annotation_dir_filename = os.path.join(pheno_annotation_ddimer_dir, pheno_annotation_filename)
pheno_annotation = pd.read_csv(pheno_annotation_dir_filename, sep = "\t", header = 0, index_col = None)

col_list = ["gengrp6", "WEIGHT_FINAL_NORM_OVERALL", "CENTER", "DDIMER", "EGFRCKDEPI", "hemoglobin_mcnc_bld_1", "hematocrit_vfr_bld_1",
            "rbc_ncnc_bld_1", "mcv_entvol_rbc_1", "mch_entmass_rbc_1", "mchc_mcnc_rbc_1", "rdw_ratio_rbc_1", "neutrophil_ncnc_bld_1",
            "lymphocyte_ncnc_bld_1", "basophil_ncnc_bld_1", "eosinophil_ncnc_bld_1", "monocyte_ncnc_bld_1", "wbc_ncnc_bld_1",
            "pmv_entvol_bld_1", "platelet_ncnc_bld_1", "lnHBA1C"]

cohort_list = ["JHS", "OMG-SCD", "PharmHU", "REDS-III-Brazil", "SAS", "HCHS_SOL", "walk-PHaSST", "WHI",
               "MESA", "GeneSTAR", "COPDGene", "CHS", "CARDIA", "BioMe", "ARIC"]
load_dir = os.path.join("..", "..", "cnv", "sample_list")
cn_pheno_anno_summary = pd.DataFrame(columns = col_list, index = cohort_list)
for cohort in cohort_list:
    if cohort == "HCHS_SOL":
        cohort_in_filename = "SOL"
    else:
        cohort_in_filename = cohort
    cohort_filename = "%s_cram.tsv"%cohort_in_filename
    cohort_dir_filename = os.path.join(load_dir, cohort_filename)
    cohort_df = pd.read_csv(cohort_dir_filename, sep = "\t", header = None, index_col = None)
    cohort_df.rename(columns = {0:"NWDID"}, inplace = True)
    cohort_pheno = cohort_df.merge(pheno_annotation, on = "NWDID", how = "inner")
    for col in col_list:
        tmp_df = cohort_pheno.loc[cohort_pheno["cohort"] == cohort, col]
        tmp_df.dropna(axis = 0, how = "any", inplace = True)
        cn_pheno_anno_summary.loc[cohort, col] = tmp_df.shape[0]
cn_pheno_anno_summary.sort_index(axis = 0, inplace = True)
cn_pheno_anno_summary_dir = os.path.join("..", "data_summary")
cn_pheno_anno_summary_filename = "overlap_cn_pheno_anno.tsv"
cn_pheno_anno_summary_dir_filename = os.path.join(cn_pheno_anno_summary_dir, cn_pheno_anno_summary_filename)
cn_pheno_anno_summary.to_csv(cn_pheno_anno_summary_dir_filename, sep = "\t", header = True, index = True)

Overlap between CN samples and annotation files

In [79]:
annotation_dir = os.path.join("..", "raw_data", "phenotype")
annotation_filename = "freeze8_sample_annot_2019-05-30.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)
annotation.dropna(axis = 0, subset = ["study"], how = "any", inplace = True)

In [88]:
cohort_list = ["JHS", "OMG-SCD", "PharmHU", "REDS-III-Brazil", "SAS", "SCD", "SOL", "walk-PHaSST", "WHI",
               "MESA", "GeneSTAR", "COPDGene", "CHS", "CARDIA", "BioMe", "ARIC"]
load_dir = os.path.join("..", "..", "cnv", "sample_list")
cohort_dir_list = [os.path.join(load_dir, "%s_cram.tsv"%cohort) for cohort in cohort_list]
commom_col = "col"
map_raw_df = annotation
save_dir = load_dir
cohort_cn_map_summary = cn_map_list(cohort_list, cohort_dir_list, map_raw_df, common_col, save_dir)
cohort_cn_map_summary_dir = os.path.join("..", "data_summary")
cohort_cn_map_summary_filename = "cohort_cn_annotation_overlap_summary.tsv"
cohort_cn_map_summary_dir_filename = os.path.join(cohort_cn_map_summary_dir, cohort_cn_map_summary_filename)
cohort_cn_map_summary.to_csv(cohort_cn_map_summary_dir_filename, sep = "\t", header = True, index = True)

/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Overlap between (Overlap between CN samples and annotation files) and blood cell traits

In [94]:
cohort_list = ["JHS", "OMG-SCD", "PharmHU", "REDS-III-Brazil", "SAS", "SOL", "walk-PHaSST", "WHI",
               "MESA", "GeneSTAR", "COPDGene", "CHS", "CARDIA", "BioMe", "ARIC"]
load_dir = os.path.join("..", "..", "cnv", "sample_list")
col_list = ["hemoglobin_mcnc_bld_1", "hematocrit_vfr_bld_1", "rbc_ncnc_bld_1", "mcv_entvol_rbc_1", "mch_entmass_rbc_1", "mchc_mcnc_rbc_1",
            "rdw_ratio_rbc_1", "neutrophil_ncnc_bld_1", "lymphocyte_ncnc_bld_1", "basophil_ncnc_bld_1", "eosinophil_ncnc_bld_1",
            "monocyte_ncnc_bld_1", "wbc_ncnc_bld_1", "pmv_entvol_bld_1", "platelet_ncnc_bld_1"]
pheno_dir = os.path.join("..", "raw_data", "phenotype")
pheno_filename = "coh03_pre.tsv"
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)
for cohort in cohort_list[:1]:
    cohort_filename = "%s_subject_id_cram.tsv"%cohort
    cohort_dir_filename = os.path.join(load_dir, cohort_filename)
    cohort_df = pd.read_csv(cohort_dir_filename, sep = "\t", header = 0, index_col = None)
    cohort_pheno = cohort_df.merge(pheno, on = "unique_subject_key", how = "inner")
display()

Compare JHS phenotype and Annotated JHS

In [112]:
jhs_pheno_filename = "jhs_basic_phenotypes_05072019.txt"
jhs_pheno_dir = os.path.join("..", "raw_data", "phenotype")
jhs_pheno_dir_filename = os.path.join(jhs_pheno_dir, jhs_pheno_filename)
jhs_pheno = pd.read_csv(jhs_pheno_dir_filename, sep = "\t", header = 0, index_col = None)
print(jhs_pheno.shape)

jhs_anno_filename = "JHS_subject_id_cram.tsv"
jhs_anno_dir = os.path.join("..", "..", "cnv", "sample_list")
jhs_anno_dir_filename = os.path.join(jhs_anno_dir, jhs_anno_filename)
jhs_anno = pd.read_csv(jhs_anno_dir_filename, sep = "\t", header = 0, index_col = None)
print(jhs_anno.shape)

jhs_pheno_anno_raw = jhs_pheno.merge(jhs_anno, on = "NWDID", how = "inner")
print(jhs_pheno_anno_raw.shape)
print("So all the annotated JHS samples having phenotypes listed in the JHS phenotype file I used at the beginning.")

jhs_pheno_anno_raw["wbc_ncnc_bld_1"] = jhs_pheno_anno_raw.shape[0] * [None]
jhs_pheno_anno_raw["neutrophil_ncnc_bld_1"] = jhs_pheno_anno_raw.shape[0] * [None]
jhs_pheno_anno_raw["lymphocyte_ncnc_bld_1"] = jhs_pheno_anno_raw.shape[0] * [None]
jhs_pheno_anno_raw["basophil_ncnc_bld_1"] = jhs_pheno_anno_raw.shape[0] * [None]
jhs_pheno_anno_raw["eosinophil_ncnc_bld_1"] = jhs_pheno_anno_raw.shape[0] * [None]
jhs_pheno_anno_raw["monocyte_ncnc_bld_1"] = jhs_pheno_anno_raw.shape[0] * [None]
jhs_pheno_anno_raw["cohort"] = jhs_pheno_anno_raw.shape[0] * ["JHS"]
col_list = ["hemoglobin_mcnc_bld_1", "hematocrit_vfr_bld_1", "rbc_ncnc_bld_1", "mcv_entvol_rbc_1", "mch_entmass_rbc_1", "mchc_mcnc_rbc_1",
            "rdw_ratio_rbc_1", "neutrophil_ncnc_bld_1", "lymphocyte_ncnc_bld_1", "basophil_ncnc_bld_1", "eosinophil_ncnc_bld_1",
            "monocyte_ncnc_bld_1", "wbc_ncnc_bld_1", "pmv_entvol_bld_1", "platelet_ncnc_bld_1", "lnHBA1C"]
jhs_pheno_anno = jhs_pheno_anno_raw[["subject_id", "unique_subject_key", "cohort", "age"] + col_list]
jhs_pheno_anno.rename(columns = {"subject_id":"SUBJECT_ID"}, inplace = True)
for col in col_list:
    jhs_pheno_anno["age_at_%s"%col] = jhs_pheno_anno["age"]
jhs_pheno_anno.drop(axis = 1, labels = "age", inplace = True)

pheno_dir = os.path.join("..", "raw_data", "phenotype")
pheno_filename = "coh02_pre.tsv"
pheno_dir_filename = os.path.join(pheno_dir, pheno_filename)
pheno = pd.read_csv(pheno_dir_filename, sep = "\t", header = 0, index_col = None)
pheno["lnHBA1C"] = pheno.shape[0] * [None]
pheno["age_at_lnHBA1C"] = pheno.shape[0] * [None]
pheno_header = list(pheno)
jhs_pheno_anno_copy = jhs_pheno_anno[pheno_header].copy()

jhs_pheno_anno_dir = os.path.join("..", "raw_data", "phenotype")
jhs_pheno_anno_filename = "jhs_basic_phenotypes_05072019_blood_cell_traits.tsv"
jhs_pheno_anno_dir_filename = os.path.join(jhs_pheno_anno_dir, jhs_pheno_anno_filename)
jhs_pheno_anno_copy.to_csv(jhs_pheno_anno_dir_filename, sep = "\t", header = True, index = False)

(3406, 36)
(3406, 3)
(3406, 38)
So all the annotated JHS samples having phenotypes listed in the JHS phenotype file I used at the beginning.


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# II. uPAR

uPAR is the name of the genes we are interested in, and APOL1 is the name of the diseases.

## 1. Convert eGFR ID to Annotation File

### 1.1 Keep AA and Hispanics for eGFR

In [54]:
egfr_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
egfr_unique_filename = "egfr03-01_unique.tsv"
egfr_unique_dir_filename = os.path.join(egfr_dir, egfr_unique_filename)
egfr_unique = pd.read_csv(egfr_unique_dir_filename, sep = "\t", header = 0, index_col = None)

In [55]:
egfr_unique["AA"] = 0
egfr_unique.loc[egfr_unique.loc[:, "race"] == 2, "AA"] = 1
egfr_sample_num = egfr_unique.shape[0]
rm_idx_list = []
for sample_i in range(egfr_sample_num):
    tmp_AA = egfr_unique.loc[sample_i, "AA"]
    tmp_ethnicity = egfr_unique.loc[sample_i, "ethnicity"]
    if tmp_AA == 0 and tmp_ethnicity == 0:
        rm_idx_list.append(sample_i)
egfr = egfr_unique.drop(labels = rm_idx_list, axis = 0)
egfr_filename = "egfr03-01_unique_AA_HS.tsv"
egfr_dir_filename = os.path.join(egfr_dir, egfr_filename)
egfr.to_csv(egfr_dir_filename, sep = "\t", header = True, index = False)

### 1.2 Map eGFR to Annotation

In [56]:
egfr_dir = os.path.join("..", "raw_data", "phenotype", "egfr_calculated")
egfr_filename = "egfr03-01_unique_AA_HS.tsv"
egfr_dir_filename = os.path.join(egfr_dir, egfr_filename)
egfr = pd.read_csv(egfr_dir_filename, sep = "\t", header = 0, index_col = None)

annotation_dir = os.path.join("..", "raw_data", "annotation")
annotation_filename = "freeze8_anno05_af02_unique02.tsv"
annotation_dir_filename = os.path.join(annotation_dir, annotation_filename)
annotation = pd.read_csv(annotation_dir_filename, sep = "\t", header = 0, index_col = None)

In [ ]:
# Here we first check if there is confliction between btc_adad and egfr03 in the columns "unique_subject_key", "cohort", "SUBJECT_ID"
pheno_col = ["unique_subject_key", "SUBJECT_ID", "cohort", "male"]
annotation_col = ["unique_subject_key", "subject_id", "study", "sex"]
pheno_prefix = "btc02-coh03_ddimer-noex_egfr03-ckd_adad01-noex"
anno_prefix = "freeze8_anno04_af02"
mapped_save_dir = os.path.join("..", "prepro_data", "phenotype")
diff_save_dir = os.path.join("..", "data_summary")
pivot_col = "unique_subject_key"
merge_how = "inner"
anno_ddimer_btc_adad_egfr = map_annotation(btc_adad_egfr, anno_ddimer, pheno_col, annotation_col, pivot_col, merge_how,
                                           pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

In [69]:
pheno_col = ["unique_subject_key", "male", "SUBJECT_ID"]
annotation_col = ["unique_subject_key", "sex", "subject_id"]
#pheno_col = ["unique_subject_key", "male"]
#annotation_col = ["unique_subject_key", "sex"]
pheno_prefix = "egfr"
anno_prefix = "freeze8_anno05_af02_unique02"
mapped_save_dir = os.path.join("..", "prepro_data", "phenotype")
diff_save_dir = os.path.join("..", "data_summary")
pivot_col = "unique_subject_key"
merge_how = "inner"

egfr_mapped = map_annotation(egfr, annotation, pheno_col, annotation_col, pivot_col, merge_how,
                             pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir)

## 2. Coding APOL1 Status

In [20]:
snp_ver = "freeze8"
snp_dir = os.path.join("..", "raw_data", "snp")
snp_id_list = ["rs71785313", "rs73885319"]
snp_dir_filename_list = [os.path.join(snp_dir, "%s_%s.raw"%(snp_ver, snp_id)) for snp_id in snp_id_list]
apol1_snp = read_snp_list_together(snp_dir_filename_list, snp_id_list)

../../eqtl/eqtl_script/function_process_data_eqtl.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp_df.dropna(axis = 0, how = "any", inplace = True)


In [21]:
sample_num = apol1_snp.shape[0]
apol1 = pd.DataFrame(columns = ["NWDID", "APOL1"])
apol1["NWDID"] = apol1_snp["NWDID"]
apol1["APOL1"] = apol1_snp["rs71785313"] + apol1_snp["rs73885319"]
apol1.loc[apol1.loc[:, "APOL1"] != 2, "APOL1"] = 0
apol1.loc[apol1.loc[:, "APOL1"] == 2, "APOL1"] = 1

In [23]:
apol1_filename = "APOL1_status.tsv"
apol1_dir = os.path.join("..", "cohort", "APOL1")
apol1_dir_filename = os.path.join(apol1_dir, apol1_filename)
apol1.to_csv(apol1_dir_filename, sep = "\t", header = True, index = False)

## 3. Correctness of rs334 freeze8 version

In [27]:
rs334_dir = os.path.join("..", "prepro_data", "snp")
rs334_freeze6a_filename = "snp_rs334_hetero.tsv"
rs334_freeze8_filename = "freeze8_rs334_hetero.tsv"

rs334_freeze6a_dir_filename = os.path.join(rs334_dir, rs334_freeze6a_filename)
rs334_freeze8_dir_filename = os.path.join(rs334_dir, rs334_freeze8_filename)

rs334_freeze6a = pd.read_csv(rs334_freeze6a_dir_filename, sep = "\t", header = 0, index_col = None)
rs334_freeze8 = pd.read_csv(rs334_freeze8_dir_filename, sep = "\t", header = 0, index_col = None)

rs334_base_NWDID = rs334_freeze6a.merge(rs334_freeze8, on = "NWDID", how = "inner")
print(rs334_base_NWDID.shape)
rs334_base_NWDID_filename = "rs334_base_NWDID.tsv"
rs334_base_NWDID_dir_filename = os.path.join(rs334_dir, rs334_base_NWDID_filename)
rs334_base_NWDID.to_csv(rs334_base_NWDID_dir_filename, sep = "\t", header = True, index = False)

rs334_base_NWDID_geno = rs334_freeze6a.merge(rs334_freeze8, on = ["NWDID", "geno"], how = "inner")
print(rs334_base_NWDID_geno.shape)

(103646, 3)
(103645, 2)


## 4. rs399145

### 4.1 Preprocess Raw rs399145 Data

In [87]:
snp_ver = "freeze8"
snp_dir = os.path.join("..", "raw_data", "snp")
snp_id_list = ["rs399145"]
snp_dir_filename_list = [os.path.join(snp_dir, "%s_%s.raw"%(snp_ver, snp_id)) for snp_id in snp_id_list]
rs399145 = read_snp_list_together(snp_dir_filename_list, snp_id_list)

../../eqtl/eqtl_script/function_process_data_eqtl.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp_df.dropna(axis = 0, how = "any", inplace = True)


In [89]:
rs399145_dir = os.path.join("..", "prepro_data", "snp")
rs399145_filename = "%s_rs399145_hetero.tsv"%snp_ver
rs399145_dir_filename = os.path.join(rs399145_dir, rs399145_filename)
rs399145.to_csv(rs399145_dir_filename, sep = "\t", header = True, index = False)

#### 4.2 (All Population) Find Two Copies of Minor Allele Individuals of rs399145 while rs334 = 1

In [24]:
snp_dir = os.path.join("..", "prepro_data", "snp")
rs399145_filename = "freeze8_rs399145_hetero.tsv"
rs399145_dir_filename = os.path.join(snp_dir, rs399145_filename)
rs399145 = pd.read_csv(rs399145_dir_filename, sep = "\t", header = 0, index_col = None)

rs334_filename = "freeze8_rs334_hetero.tsv"
rs334_dir_filename = os.path.join(snp_dir, rs334_filename)
rs334 = pd.read_csv(rs334_dir_filename, sep = "\t", header = 0, index_col = None)

In [35]:
rs334_rs399145 = rs334.merge(rs399145, on = "NWDID", how = "inner")
rs334_heter_rs399145 = rs334_rs399145.loc[rs334_rs399145.loc[:, "rs334"] == 1, :]
rs334_heter_rs399145_homo = rs334_heter_rs399145.loc[rs334_heter_rs399145.loc[:, "rs399145"] == 2, :]
print(rs334_heter_rs399145_homo.shape)
rs334_heter_rs399145_homo_dir = os.path.join("..", "data_summary")
rs334_heter_rs399145_homo_filename = "freeze8_rs334-heter_rs399145-2.tsv"
rs334_heter_rs399145_homo_dir_filename = os.path.join(rs334_heter_rs399145_homo_dir, rs334_heter_rs399145_homo_filename)
rs334_heter_rs399145_homo.to_csv(rs334_heter_rs399145_homo_dir_filename, sep = "\t", header = True, index = False)

(52, 3)


In [36]:
rs334_rs399145_dir = os.path.join("..", "cohort", "APOL1", "ready_data")
rs334_rs399145_filename = "common_rs399145-rs334.tsv"
rs334_rs399145_dir_filename = os.path.join(rs334_rs399145_dir, rs334_rs399145_filename)
rs334_rs399145 = pd.read_csv(rs334_rs399145_dir_filename, sep = "\t", header = 0, index_col = None)

rs334_heter_rs399145 = rs334_rs399145.loc[rs334_rs399145.loc[:, "rs334"] == 1, :]
rs334_heter_rs399145_homo = rs334_heter_rs399145.loc[rs334_heter_rs399145.loc[:, "rs399145"] == 2, :]
print(rs334_heter_rs399145_homo.shape)

rs334_heter_rs399145_homo_dir = os.path.join("..", "data_summary")
rs334_heter_rs399145_homo_filename = "freeze8_rs334-heter_rs399145-2_used.tsv"
rs334_heter_rs399145_homo_dir_filename = os.path.join(rs334_heter_rs399145_homo_dir, rs334_heter_rs399145_homo_filename)
rs334_heter_rs399145_homo.to_csv(rs334_heter_rs399145_homo_dir_filename, sep = "\t", header = True, index = False)

(15, 4)


In [45]:
snp_ver = "freeze8"
snp_dir = os.path.join("..", "raw_data", "snp")
snp_id_list = ["rs2302524", "rs2633317", "rs4251805", "rs4760", "rs73935023"]
snp_dir_filename_list = [os.path.join(snp_dir, "%s_%s.raw"%(snp_ver, snp_id)) for snp_id in snp_id_list]
snp_list = read_snp_list_each(snp_dir_filename_list, snp_id_list)

../../eqtl/eqtl_script/function_process_data_eqtl.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snp_df.dropna(axis = 0, how = "any", inplace = True)


In [59]:
ddimer_filename = "TOPMED_HarmonizedPhenotypes_DDIMER_21MAY2019.csv"
ddimer_dir = os.path.join("..", "raw_data", "phenotype")
ddimer_dir_filename = os.path.join(ddimer_dir, ddimer_filename)
ddimer_raw = pd.read_csv(ddimer_dir_filename, sep = ",", header = 0, index_col = None)
ddimer = ddimer_raw.loc[:, ["sample.id", "STUDY", "sex", "AGE_DDIMER", "DDIMER", "sample_remove_DDIMER"]]
ddimer.dropna(axis = "index", how = "any", inplace = True)
ddimer.rename(columns = {"sample.id":"NWDID", "STUDY":"cohort", "AGE_DDIMER":"age_at_DDIMER"}, inplace = True)
sex_dict = {"M":1, "F":0}
ddimer.replace({"sex": sex_dict}, inplace = True)
save_dir = os.path.join("..", "raw_data", "phenotype")
ddimer_filename = "DDIMER_21MAY2019_complete_useful_col.tsv"
ddimer_dir_filename = os.path.join(save_dir, ddimer_filename)
ddimer.to_csv(ddimer_dir_filename, sep = "\t", header = True, index = False)
print(duplicates_num(ddimer, "NWDID"))
print("There is no duplicates in NWDID in DDIMER I processed.")

0
There is no duplicates in NWDID in DDIMER I processed.


/nas/longleaf/home/minzhi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
def map_deprecation_list(pheno, annotation, pheno_col, annotation_col, pivot_col, pheno_prefix, anno_prefix, save_dir):
    pheno_annotatio_dif = annotation.merge(pheno, left_on = pheno_col[0], right_on = annotation_col[0], how = "outer")
    depre_sample_list = []
    pivot_df = pheno_annotatio_dif[[pivot_col]]
    for pheno_col_i, annotation_col_i in zip(pheno_col[1:], annotation_col[1:]):
        if pheno_col_i == annotation_col_i:
            annotation_col_i = "%s_x"%annotation_col_i
            pheno_col_i = "%s_y"%pheno_col_i
        col_dif = pheno_annotatio_dif[[pivot_col, annotation_col_i, pheno_col_i]]
        tmp_depre_sample_list, df_col_i = map_deprecation(col_dif, pheno_col_i, annotation_col_i,
                                                          pivot_col, pheno_prefix, anno_prefix, save_dir)
        pivot_df = pivot_df.merge(df_col_i, on = pivot_col, how = "inner")
        depre_sample_list = depre_sample_list + tmp_depre_sample_list
    return depre_sample_list, pivot_df

In [282]:
df3 = pd.DataFrame(df1.apply(func, axis = 1), columns = ['A'])
display(df3)

,A
0,0.0
1,0.0
2,1.0
3,1.0
4,0.0
5,0.0
6,1.0
7,0.0
8,1.0
9,0.0


In [297]:
df4 = pd.DataFrame({'A': [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, None, 0], 'B': [0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]})
display(df4)
df4.iloc[:, 0] = df4.apply(func, axis = 1)
display(df4)

,A,B
0,0.0,0
1,0.0,1
2,1.0,1
3,1.0,1
4,0.0,0
5,0.0,0
6,1.0,1
7,1.0,0
8,1.0,1
9,0.0,0


,A,B
0,0.0,0
1,0.0,1
2,1.0,1
3,1.0,1
4,0.0,0
5,0.0,0
6,1.0,1
7,1.0,0
8,1.0,1
9,0.0,0


In [298]:
sample_lost = df4[df4.iloc[:, 0].isnull()].index.tolist()
print(sample_lost)

[]


In [46]:
df0 = pd.DataFrame({'A': ["a", "g", "c", "d", "k"],
                    'B': [3, 4, 5, 6, 7]})
df1 = pd.DataFrame({'A': ["a", "b", "c", "d", "e", "f", "g", "h", "i"],
                    'B': [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
                    'C': ["nice", "good", "exce", "ama", "awe", np.nan, "perf", "terri", "well"]})
display(df0)
display(df1)
df1 = df1.set_index('A')
df0 = df0.set_index('A')
df3 = df1.fillna(df0)
df4 = df1.combine_first(df0)
display(df3)
display(df4)
df3 = df3.reset_index()
display(df3)

,A,B
0,a,3
1,g,4
2,c,5
3,d,6
4,k,7


,A,B,C
0,a,NaN,nice
1,b,NaN,good
2,c,NaN,exce
3,d,NaN,ama
4,e,NaN,awe
5,f,NaN,NaN
6,g,NaN,perf
7,h,NaN,terri
8,i,NaN,well


,B,C
A,,
a,3.0,nice
b,NaN,good
c,5.0,exce
d,6.0,ama
e,NaN,awe
f,NaN,NaN
g,4.0,perf
h,NaN,terri
i,NaN,well


,B,C
A,,
a,3.0,nice
b,NaN,good
c,5.0,exce
d,6.0,ama
e,NaN,awe
f,NaN,NaN
g,4.0,perf
h,NaN,terri
i,NaN,well


,A,B,C
0,a,3.0,nice
1,b,NaN,good
2,c,5.0,exce
3,d,6.0,ama
4,e,NaN,awe
5,f,NaN,NaN
6,g,4.0,perf
7,h,NaN,terri
8,i,NaN,well


In [ ]:
df2 = df0.merge(df1, left_on = ['B'], right_on = ['C'], how = "inner")
display(df2)

In [58]:
def map_annotation(pheno, annotation, pheno_col, annotation_col, pivot_col, how_merge, pheno_prefix, anno_prefix, mapped_save_dir, diff_save_dir):
    pheno_mapped = annotation.merge(pheno, left_on = annotation_col[0], right_on = pheno_col[0], how = merge_how)
    col_num = len(pheno_col[1:])
    for col_i in range(1, col_num + 1):
        if pheno_col[col_i] != annotation_col[col_i]:
            pheno_mapped.drop(axis = 1, columns = [pheno_col[col_i], annotation_col[col_i]], inplace = True)
        else:
            tmp_pheno_col = "%s_y"%pheno_col[col_i]
            tmp_annotation_col = "%s_x"%annotation_col[col_i]
            pheno_mapped.drop(axis = 1, columns = [tmp_pheno_col, tmp_annotation_col], inplace = True)
    if len(annotation_col) > 1:
        depre_sample_list, pivot_col_df = map_deprecation_list(pheno, annotation, pheno_col, annotation_col,
                                                               pivot_col, pheno_prefix, anno_prefix, diff_save_dir)
        if depre_sample_list != []:
            pheno_mapped_raw = pheno_mapped.copy()
            del pheno_mapped
            pheno_mapped = pheno_mapped_raw[~pheno_mapped_raw[pivot_col].isin(depre_sample_list)]
        pheno_mapped = pheno_mapped.merge(pivot_col_df, on = pivot_col, how = "inner")
    pheno_mapped_filename = "%s_%s.tsv"%(anno_prefix, pheno_prefix)
    pheno_mapped_dir_filename = os.path.join(mapped_save_dir, pheno_mapped_filename)
    pheno_mapped.to_csv(pheno_mapped_dir_filename, sep = "\t", header = True, index = False)
    return pheno_mapped

In [60]:
def map_deprecation(df, pheno_col, annotation_col, pivot_col, pheno_prefix, anno_prefix, save_dir):
    df_pivot = df[[pivot_col]]
    df_col = df[[pheno_col, annotation_col]]
    df_col.iloc[:, 0] = df_col.apply(func, axis = 1)
    df_col.iloc[:, 1] = df_col.apply(func, axis = 1)
    del df
    df = pd.concat([df_pivot, df_col], axis = 1)
    depre_sample_index_list = df[df.iloc[:, 0].isnull()].index.tolist()
    if depre_sample_index_list == []:
        depre_sample_list = []
    else:
        depre_sample_list = df.loc[depre_sample_index_list, pivot_col].values.reshape(1, -1).tolist()[0]
    df.dropna(axis = 0, how = "any", inplace = True)
    tmp_compare = df[pheno_col].eq(df[annotation_col], axis = 0)
    df_dif = df[tmp_compare == False]
    df_cons = df[tmp_compare == True]
    if annotation_col.split('_x')[-1] == "":
        annotation_col_propagate = annotation_col.split('_x')[0]
        df_cons.rename(columns = {annotation_col:annotation_col_propagate}, inplace = True)
    else:
        annotation_col_propagate = annotation_col
    df_cons_propagate = df_cons.loc[:, [pivot_col, annotation_col_propagate]]
    if df_dif.shape[0] != 0:
        print("%s is inconsistent."%pheno_col)
        df_dif_filename = "%s_%s_%s.tsv"%(anno_prefix, pheno_prefix, annotation_col)
        df_dif_dir_filename = os.path.join(save_dir, df_dif_filename)
        df_dif.to_csv(df_dif_dir_filename, sep = "\t", header = True, index = False)
        depre_sample_list = df_dif[[pivot_col]].values.reshape(1, -1).tolist()[0]
    return depre_sample_list, df_cons_propagate

In [59]:
def map_deprecation_list(pheno, annotation, pheno_col, annotation_col, pivot_col, pheno_prefix, anno_prefix, save_dir):
    pheno_annotatio_dif = annotation.merge(pheno, left_on = pheno_col[0], right_on = annotation_col[0], how = "outer")
    depre_sample_list = []
    pivot_df = pheno_annotatio_dif[[pivot_col]]
    for pheno_col_i, annotation_col_i in zip(pheno_col[1:], annotation_col[1:]):
        if pheno_col_i == annotation_col_i:
            annotation_col_i = "%s_x"%annotation_col_i
            pheno_col_i = "%s_y"%pheno_col_i
        col_dif = pheno_annotatio_dif[[pivot_col, annotation_col_i, pheno_col_i]]
        tmp_depre_sample_list, df_col_i = map_deprecation(col_dif, pheno_col_i, annotation_col_i,
                                                          pivot_col, pheno_prefix, anno_prefix, save_dir)
        pivot_df = pivot_df.merge(df_col_i, on = pivot_col, how = "inner")
        depre_sample_list = depre_sample_list + tmp_depre_sample_list
    return depre_sample_list, pivot_df